# Récupérer les projets de recherche ANR portés par son établissememnt
**A l'aide des identifiants RNSR**

0. télécharger les jeux de données


1. importer les données des partenaires, comprendre le schéma de données


2. récupérer les RNSRs de son établissement via l'API de AureHAL Structure
    - comprendre l'API AureHAL Structure
    - quel champs utiliser pour identifier les sous structures de son étab. ? 
    - extraire un tableau de vos structures avec les donénes docid,name_s,type_s,valid_s,rnsr_s
    - _à la main_ renseigner dans AureHAL les RNSRs manquants
    - placer les RNSRs dans une liste
    - nettoyer les données
   
   
3. filtrer les données ANR des partenaires avec les RNSR récoltés
    - exporter un csv

----


## 0. télécharger les jeux de données
[https://www.data.gouv.fr/en/datasets/anr-01-projets-anr-dos-detail-des-projets-et-des-partenaires/](https://www.data.gouv.fr/en/datasets/anr-01-projets-anr-dos-detail-des-projets-et-des-partenaires/)

attention le schéma de données peut évoluer

## 1. importer les données des partenaires

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/anr-dos-depuis-2010-projets-finances-20210826-partenaires.csv", sep = ';')
df.sample(10)

,Projet.Code_Decision_ANR,Projet.Acronyme,Projet.Partenaire.Code_Decision_ANR,Projet.Partenaire.Est_coordinateur,Projet.Partenaire.Nom_organisme,Projet.Partenaire.Categorie_organisme,Projet.Partenaire.Code_RNSR,Projet.Partenaire.Responsable_scientifique.Nom,Projet.Partenaire.Responsable_scientifique.Prenom,Projet.Partenaire.Adresse.Ville,Projet.Partenaire.Adresse.Region,Projet.Partenaire.Adresse.Pays
25958,ANR-17-CE11-0024,H4translation,ANR-17-CE11-0024-01,True,Architecture et Réactivité de lARN,Laboratoire public,199317388K,ERIANI,Gilbert,Strasbourg,Grand Est,France
2076,ANR-20-COVI-0034,CoCo,ANR-20-COVI-0034-01,True,Observatoire sociologique du changement,Laboratoire public,198821638B,RECCHI,Ettore,PARIS,Ile-de-France,France
62072,ANR-10-BIOT-0006,DbaitTherap,ANR-10-BIOT-0006-04,False,UNIVERSITE DE VERSAILLES - SAINT-QUENTIN - EN ...,Laboratoire public,NaN,SAIAG,Philippe,Boulogne-Billancourt,Île-de-France,France
33311,ANR-15-CE38-0009,INVENTHEA,ANR-15-CE38-0009-02,False,Laboratoire de Mécanique et dAcoustique,Laboratoire public,199617659S,GUILLEMAIN,Philippe,Marseille,Provence-Alpes-Côte d'Azur,France
34147,ANR-15-CE24-0028,OLOVIA,ANR-15-CE24-0028-04,False,UNITY SEMICONDUCTOR,PME,NaN,ALLIATA,Dario,Montbonnot-Saint-Martin,Auvergne-Rhône-Alpes,France
49039,ANR-12-BSV5-0003,MAX4US,ANR-12-BSV5-0003-02,False,SPCSI,Divers public,NaN,BERTHELOT,Thomas,Gif-sur-Yvette,Île-de-France,France
15253,ANR-19-CE05-0032,UPGEO,ANR-19-CE05-0032-05,False,GEORESSOURCES & ENVIRONNEMENT,Laboratoire public,201119423U,BOURILLOT,Raphaël,Pessac,Nouvelle-Aquitaine,France
61049,ANR-10-BLAN-0946,FELINS,ANR-10-BLAN-0946-01,True,CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE -...,Divers public,NaN,CANIONI,Lionel,Talence,Nouvelle-Aquitaine,France
6939,ANR-20-CE13-0027,INFER-ANTH,ANR-20-CE13-0027-01,True,REPRODUCTION ET DEVELOPPEMENT DES PLANTES,Laboratoire public,200317442A,SCUTT,Charles,LYON CEDEX 07,Auvergne-Rhône-Alpes,France
33225,ANR-15-CE39-0016,VIORAMIL,ANR-15-CE39-0016-02,False,Centre de Recherche sur les Médiations,Laboratoire public,200114752K,WALTER,Jacques,Metz,Grand Est,France


In [9]:
# aperçu taux de complétude des codes RNSR
print(f"nombre de partenaires {len(df)}")
print(f"nombre rnsr manquants {len(df[ df['Projet.Partenaire.Code_RNSR'].isna()] )}")

nombre de partenaires 62320
nombre rnsr manquants 31123


## 2. Récupérer les RNSRs de son établissement via l'API AureHAL Structure

In [10]:
# documentation https://api.archives-ouvertes.fr/docs

In [11]:
import requests, json

In [12]:
# tester l'extraction de rnsr
# u orleans : 300297
ex = requests.get("https://api.archives-ouvertes.fr/ref/structure/?q=parentDocid_i:300297&fl=rnsr_s&rows=15")
ex.json()

{'response': {'numFound': 152,
  'start': 0,
  'docs': [{},
   {'rnsr_s': ['199213388S']},
   {},
   {},
   {},
   {},
   {},
   {'rnsr_s': ['199112390L']},
   {},
   {'rnsr_s': ['200615274F']},
   {'rnsr_s': ['199213386P']},
   {},
   {'rnsr_s': ['201822709K']},
   {'rnsr_s': ['200817616T']},
   {}]}}

In [17]:
struct_raw = requests.get("https://api.archives-ouvertes.fr/ref/structure/?q=parentDocid_i:300297&fq=valid_s:(VALID OR OLD)&fl=docid,name_s,type_s,valid_s,rnsr_s&rows=10000")
# degré de précision : réduire aux structures valides ou OLD ...
all_struct = struct_raw.json()["response"]["docs"]
df_struct = pd.DataFrame(all_struct)
# df_struct.to_csv pour extraire un csv ...

In [19]:
# but :  recupérer tous les rnsr
## a_ retirer tous les NaN
df_rnsr = df_struct[ df_struct["rnsr_s"].notna()]
df_rnsr.sample(3)

,docid,name_s,type_s,valid_s,rnsr_s
16,1002450,Laboratoire de Biologie des Ligneux et des Gra...,laboratory,VALID,[199213387R]
51,955,Groupe de recherches sur l'énergétique des mil...,laboratory,OLD,[199612379C]
33,551843,Mathématiques appliquées et physique mathémati...,laboratory,OLD,[199412399H]


In [20]:
## b_ les placer dans une liste
my_rnsr = df_rnsr["rnsr_s"].tolist()
my_rnsr[:4]

[['199213388S'], ['199112390L'], ['200615274F'], ['199213386P']]

In [21]:
## c_ la nettoyer
my_rnsr_clean = [item[0] for item in my_rnsr]
print(f"nb rnsr recupéré {len(my_rnsr_clean)}")

nb rnsr recupéré 57


## 3. Filtrer les données ANR des partenaires avec les RNSR récoltés

In [24]:
df_part = pd.read_csv("anr-dos-depuis-2010-projets-finances-20210826-partenaires.csv", sep = ';')
df_part.sample(5)

,Projet.Code_Decision_ANR,Projet.Acronyme,Projet.Partenaire.Code_Decision_ANR,Projet.Partenaire.Est_coordinateur,Projet.Partenaire.Nom_organisme,Projet.Partenaire.Categorie_organisme,Projet.Partenaire.Code_RNSR,Projet.Partenaire.Responsable_scientifique.Nom,Projet.Partenaire.Responsable_scientifique.Prenom,Projet.Partenaire.Adresse.Ville,Projet.Partenaire.Adresse.Region,Projet.Partenaire.Adresse.Pays
18492,ANR-18-CE34-0001,CREATIvE,ANR-18-CE34-0001-02,False,TOXICOLOGIE PHARMACOLOGIE ET SIGNALISATION CEL...,Laboratoire public,200616420B,COUMOUL,Xavier,Paris,Île-de-France,France
3538,ANR-20-CE42-0003,3DPolariSR,ANR-20-CE42-0003-03,False,Laboratoire des biomolécules,Laboratoire public,200919226N,GAUTIER,Arnaud,PARIS,Ile-de-France,France
18392,ANR-18-CE36-0003,DERVI,ANR-18-CE36-0003-02,False,PLEIADE,Laboratoire public,201420821A,DEMARTINI,Anne-Emmanuelle,Villetaneuse,Île-de-France,France
41588,ANR-13-SAMA-0007,CANNADO,ANR-13-SAMA-0007-03,False,Institut de Neurobiologie de la Mediterranée,Laboratoire public,200819256A,ROBBE,David,Marseille,Provence-Alpes-Côte d'Azur,France
57514,ANR-10-RFCS-0008,MobiNiCSH,ANR-10-RFCS-0008-01,True,COMMISSARIAT A LENERGIE ATOMIQUE ET AUX ENERGI...,Divers public,NaN,ROMEO,Paul- Henri,Fontenay-aux-Roses,Ile-de-France,France


In [25]:
my_project = df_part[ df_part["Projet.Partenaire.Code_RNSR"].isin(my_rnsr_clean)]
print(f" nb projets identifiés {len(my_project)}")
# exporter en csv : my_project.to_csv("projets_anr_identifies.csv", index = False)

 nb projets identifiés 211


----

## Discussion

Maintenant que vous avez identifié vos projets, que pouvez-faire pour développer la science ouverte ? 

(au moins 5 possibilités ai-je eu en tête)

---

## Extra : compléter par une recherche sur le nom de l'étab.
- identifier les proejts de recherche avec le nom d'un étab.
- ajouter les projets qui n'étaient pas déjà présents avec l'identification par RNSR

In [27]:
# réduire les données aux projets où des noms d'organisme sont présents
org_name = df_part[ df_part["Projet.Partenaire.Nom_organisme"].notna()]

In [29]:
# faire une regex pour le nom de votre étab.
name_filter = org_name["Projet.Partenaire.Nom_organisme"].str.contains("universit. dorl.ans", case = False, regex=True)
my_project2 = org_name[name_filter].reset_index()
my_project2.sample(5)

,index,Projet.Code_Decision_ANR,Projet.Acronyme,Projet.Partenaire.Code_Decision_ANR,Projet.Partenaire.Est_coordinateur,Projet.Partenaire.Nom_organisme,Projet.Partenaire.Categorie_organisme,Projet.Partenaire.Code_RNSR,Projet.Partenaire.Responsable_scientifique.Nom,Projet.Partenaire.Responsable_scientifique.Prenom,Projet.Partenaire.Adresse.Ville,Projet.Partenaire.Adresse.Region,Projet.Partenaire.Adresse.Pays
9,53153,ANR-11-INSE-0006,MAGIC-SPS,ANR-11-INSE-0006-04,False,UNIVERSITE DORLEANS,Laboratoire public,NaN,RAMDANI,Nacim,Bourges,NaN,NaN
1,773,ANR-20-THIA-0017,AI.i0 PhD Fellowship,ANR-20-THIA-0017-01,True,Université dOrléans,Divers public,NaN,VRAIN,Christel,ORLEANS,Centre-Val de Loire,France
0,772,ANR-20-THIA-0017,AI.i0 PhD Fellowship,ANR-20-THIA-0017-01,True,Université dOrléans,Divers public,NaN,VRAIN,Christel,ORLEANS,NaN,France
3,33027,ANR-15-FRAL-0004,SegCor,ANR-15-FRAL-0004-02,False,"Université dOrléans-CNRS, Laboratoire Ligérien...",Laboratoire public,201220244G,SKROVEC,Marie,Orléans,NaN,NaN
16,56665,ANR-10-VPTT-0002,ICAMDAC,ANR-10-VPTT-0002-03,False,UNIVERSITE DORLEANS,Laboratoire public,NaN,FOUCHER,Fabrice,Orléans,NaN,NaN


In [30]:
## réduire aux projects qui n'ont pas déjà été trouvés par les RNSR
project_by_orgname = my_project2[ ~my_project2["Projet.Code_Decision_ANR"].isin(
my_project["Projet.Code_Decision_ANR"].tolist()) ]
## nota : d'autres solutions existent pour réaliser ce procédé
project_by_orgname.sample(7)

,index,Projet.Code_Decision_ANR,Projet.Acronyme,Projet.Partenaire.Code_Decision_ANR,Projet.Partenaire.Est_coordinateur,Projet.Partenaire.Nom_organisme,Projet.Partenaire.Categorie_organisme,Projet.Partenaire.Code_RNSR,Projet.Partenaire.Responsable_scientifique.Nom,Projet.Partenaire.Responsable_scientifique.Prenom,Projet.Partenaire.Adresse.Ville,Projet.Partenaire.Adresse.Region,Projet.Partenaire.Adresse.Pays
22,61100,ANR-10-BLAN-0930,PAMPA,ANR-10-BLAN-0930-03,False,UNIVERSITE DORLEANS,Laboratoire public,NaN,ROBERT,Eric,Orléans,NaN,NaN
8,52691,ANR-11-JS09-0010,INDIGO,ANR-11-JS09-0010-01,True,UNIVERSITE DORLEANS,Laboratoire public,NaN,MIKIKIAN,Maxime,Orléans,NaN,NaN
1,773,ANR-20-THIA-0017,AI.i0 PhD Fellowship,ANR-20-THIA-0017-01,True,Université dOrléans,Divers public,NaN,VRAIN,Christel,ORLEANS,Centre-Val de Loire,France
12,55532,ANR-11-BS09-0029,PLASBorDDiAM,ANR-11-BS09-0029-02,False,UNIVERSITE DORLEANS,Laboratoire public,NaN,BRAULT,Pascal,Orléans,NaN,NaN
20,59725,ANR-10-CESA-0007,Neuropest,ANR-10-CESA-0007-02,False,UNIVERSITE DORLEANS,Laboratoire public,NaN,MONTÉCOT-DUBOURG,Céline,Orléans,NaN,NaN
17,56968,ANR-10-TECS-0004,CancerSensor,ANR-10-TECS-0004-01,True,UNIVERSITE DORLEANS,Laboratoire public,NaN,AGROFOGLIO,Luigi,Orléans,NaN,NaN
19,58551,ANR-10-INTB-0205,PAPDAS,ANR-10-INTB-0205-01,True,UNIVERSITE DORLEANS,Laboratoire public,NaN,DABROWSKI,Frédéric,Orléans,NaN,NaN


In [31]:
# concaténer les deux tableaux des projets de votre étab. 
all_my_project = pd.concat([my_project, project_by_orgname])
print("nb de projet ANR total identifié", len(all_my_project))

nb de projet ANR total identifié 226
